## Revise train data judgement



In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
input_dir = "/content/drive/MyDrive/07_Competition/signate-471/data/"
output_dir = "/content/drive/MyDrive/07_Competition/signate-471/log/"
submission_dir = "/content/drive/MyDrive/07_Competition/signate-471/submission/"
model_dir = "/content/drive/MyDrive/07_Competition/signate-471/model_bin/"
pred_dir = "/content/drive/MyDrive/07_Competition/signate-471/pred/"

In [3]:
!pip install transformers pycld2
!pip install nlplot 

     |████████████████████████████████| 2.8 MB 4.3 MB/s 
     |████████████████████████████████| 41.4 MB 96 kB/s 
     |████████████████████████████████| 50 kB 7.6 MB/s 
     |████████████████████████████████| 636 kB 63.2 MB/s 
     |████████████████████████████████| 895 kB 62.1 MB/s 
     |████████████████████████████████| 3.3 MB 70.8 MB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834340 sha256=57c4b342ead32aa0686be74d6cc64b9f9ab896d7665664e9aeefe09dd5f5033d
  Stored in directory: /root/.cache/pip/wheels/ed/e4/58/ed2e9f43c07d617cc81fe7aff0fc6e42b16c9cf6afe960b614
Successfully built pycld2
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 967 kB 4.3 MB/s 
     |████████████████████████████████| 23.9 MB 1.3 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plo

In [4]:
import os
import math
import random
import pandas as pd
import numpy as np
from glob import glob
import gc
gc.enable()

import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim as optim
from torch.optim.optimizer import Optimizer
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader, SequentialSampler, RandomSampler

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import fbeta_score

from IPython.display import clear_output
from tqdm import tqdm, trange
import spacy
from sklearn.metrics.pairwise import cosine_similarity

import re
import nltk
from nltk.corpus import stopwords
from scipy.optimize import minimize, minimize_scalar
import regex
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# 元ファイルの読み込みと前処理

In [5]:
pd.set_option("display.max_colwidth", 50)
train = pd.read_csv(input_dir + "train.csv")
test = pd.read_csv(input_dir + "test.csv")
sub = pd.read_csv(input_dir + "sample_submit.csv", header=None)
sub.columns = ["id", "judgement"]

In [8]:
train_1 = train[train.judgement==1].reset_index(drop=True)

In [19]:
len(train_1)

632

In [16]:
sum(train_1.abstract.isnull())

122

In [17]:
sum(train_1.title.isnull())

0

In [12]:
train_1.iloc[0].abstract

"BACKGROUND: Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) infection has become a public health emergency of international concern. SARS-CoV-2 RNA detection is the diagnostic criterion for coronavirus disease 2019 (COVID-19). Nevertheless, RNA detection has many limitations, such as being time-consuming and cost-prohibitive, and it must be performed in specialized laboratories. Virus antibody detection is a routine method for screening for multiple viruses, but data about SARS-CoV-2 antibody detection are limited. METHOD: Throat swabs and blood were collected from 67 suspected SARS-CoV-2 infection patients at the Affiliated Hospital of Zunyi Medical University and Zunyi Fourth People's Hospital isolated observation departments. Throat swab samples were subjected to SARS-CoV-2 RNA detection by real-time PCR. Blood was used subjected to SARS-CoV-2 IgG/IgM detection by an enzyme-linked immunosorbent assay (ELISA) and gold immunochromatography assay (GICA). Blood underwent C

In [13]:
train_1.iloc[1].abstract

"Diagnostic accuracy in FDG-PET imaging highly depends on the operating procedures. In this clinical study on dementia  we compared the diagnostic accuracy at a single-subject level of a) Clinical Scenarios  b) Standard FDG Images and c) Statistical Parametrical (SPM) Maps generated via a new optimized SPM procedure. We evaluated the added value of FDG-PET  either Standard FDG Images or SPM Maps  to Clinical Scenarios. In 88 patients with neurodegenerative diseases (Alzheimer's Disease-AD  Frontotemporal Lobar Degeneration-FTLD  Dementia with Lewy bodies-DLB and Mild Cognitive Impairment-MCI)  9 neuroimaging experts made a forced diagnostic decision on the basis of the evaluation of the three types of information. There was also the possibility of a decision of normality on the FDG-PET images. The clinical diagnosis confirmed at a long-term follow-up was used as the gold standard. SPM Maps showed higher sensitivity and specificity (96% and 84%)  and better diagnostic positive (6.8) and

In [18]:
train_1.iloc[3].abstract

'As the Coronavirus disease 2019 continues to circulate, testing strategies are of the utmost importance. Given national shortages of testing supplies, personal protective equipment, and other hospital resources, diagnostic stewardship is necessary to aid in resource management. We report the low utility of serial testing in a low prevalence setting.'

In [20]:
train_1.iloc[4].abstract

'From late December 2019 COVID-19 (Coronavirus infectious disease 19) epidemics spread from Wuhan, China, to all over the world, including Italy. This article is protected by copyright. All rights reserved. Copyright This article is protected by copyright. All rights reserved.'

In [22]:
train_1.iloc[6].abstract

'Nowadays, automatic disease detection has become a crucial issue in medical science due to rapid population growth. An automatic disease detection framework assists doctors in the diagnosis of disease and provides exact, consistent, and fast results and reduces the death rate. Coronavirus (COVID-19) has become one of the most severe and acute diseases in recent times and has spread globally. Therefore, an automated detection system, as the fastest diagnostic option, should be implemented to impede COVID-19 from spreading. This paper aims to introduce a deep learning technique based on the combination of a convolutional neural network (CNN) and long short-term memory (LSTM) to diagnose COVID-19 automatically from X-ray images. In this system, CNN is used for deep feature extraction and LSTM is used for detection using the extracted feature. A collection of 4575 X-ray images, including 1525 images of COVID-19, were used as a dataset in this system. The experimental results show that our

In [24]:
train_1.iloc[8].abstract

"Background 123 I-FP-CIT SPECT imaging is a reliable method to assess presynaptic dopaminergic pathways in degenerative parkinsonisms and dementia with Lewy bodies (DLB). Methods We aimed at examining sensitivity of combined visual and semi-quantitative 123 I-FP-CIT SPECT analyses in a prospective cohort of subjects with DLB and degenerative parkinsonisms - Parkinson's disease (PD)  multiple system atrophy (MSA)  corticobasal syndrome (CBS) and progressive supranuclear palsy (PSP) to determine prevalence and clinical significance of scans without evidence of dopaminergic deficit (SWEDD). 372 scans performed from 2013 to 2016 with the same SPECT acquisition and processing protocol were analyzed. We identified 155 patients with degenerative parkinsonism and 53 with DLB. Diagnoses relied on validated clinical criteria for each condition. Semi-quantitative assessment was based on previously established local reference limits (including striatal volumes of interest uptake  caudate-to-putame

In [ ]:
train["title_abst"] = train["title"] + train["abstract"]
train["title_abst"].fillna(train["title"], inplace=True)

test["title_abst"] = test["title"] + test["abstract"]
test["title_abst"].fillna(test["title"], inplace=True)


train.head()

,id,title,abstract,judgement,title_abst
0,0,One-year age changes in MRI brain volumes in o...,Longitudinal studies indicate that declines in...,0,One-year age changes in MRI brain volumes in o...
1,1,Supportive CSF biomarker evidence to enhance t...,The present study was undertaken to validate t...,0,Supportive CSF biomarker evidence to enhance t...
2,2,Occurrence of basal ganglia germ cell tumors w...,Objective: To report a case series in which ba...,0,Occurrence of basal ganglia germ cell tumors w...
3,3,New developments in diagnosis and therapy of C...,The etiology and pathogenesis of idiopathic ch...,0,New developments in diagnosis and therapy of C...
4,4,Prolonged shedding of SARS-CoV-2 in an elderly...,NaN,0,Prolonged shedding of SARS-CoV-2 in an elderly...


In [ ]:
def remove_non_alphabetic(texts):
    text_list=[]

    for text in texts: 
        text = re.sub(r"[^a-zA-Z]", " ", text)
        text = text.replace("  ", " ")
        text_list.append(text)
    return text_list


nltk.download('stopwords')
stop_words = stopwords.words('english')

def clean_stopword(text_list):
    texts_temp= [[word for word in text.lower().split() if word not in stop_words] for text in text_list]
    return texts_temp


def split_copyright(texts):

    text_list=[]

    for text in texts:    

        if "Copyright" in text:
            text = text.split('Copyright')[0]
            text_list.append(text)
        else:
            text_list.append(text)
    return text_list

nlp = spacy.load('en', disable=['parser', 'ner'])

def lemmatization(texts_list, allowed_postags=['PROPN','VERB','NOUN','ADJ', 'ADV']):
   
    texts_pre=[]

    for sent in texts_list:
        doc = nlp(" ".join(sent)) 
        texts_pre.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_pre

def preprocess_text(text):
    text = remove_non_alphabetic(text)
    text = split_copyright(text)
    text = clean_stopword(text)
    text = lemmatization(text)
    
    return text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train["title_abst"] = train["title"] + train["abstract"]
train["title_abst"].fillna(train["title"], inplace=True)

test["title_abst"] = test["title"] + test["abstract"]
test["title_abst"].fillna(test["title"], inplace=True)

train.head()

,id,title,abstract,judgement,title_abst
0,0,One-year age changes in MRI brain volumes in o...,Longitudinal studies indicate that declines in...,0,One-year age changes in MRI brain volumes in o...
1,1,Supportive CSF biomarker evidence to enhance t...,The present study was undertaken to validate t...,0,Supportive CSF biomarker evidence to enhance t...
2,2,Occurrence of basal ganglia germ cell tumors w...,Objective: To report a case series in which ba...,0,Occurrence of basal ganglia germ cell tumors w...
3,3,New developments in diagnosis and therapy of C...,The etiology and pathogenesis of idiopathic ch...,0,New developments in diagnosis and therapy of C...
4,4,Prolonged shedding of SARS-CoV-2 in an elderly...,NaN,0,Prolonged shedding of SARS-CoV-2 in an elderly...


In [ ]:
title_abst_train= train.title_abst
title_abst_test = test.title_abst

title_abst_train= preprocess_text(title_abst_train)
title_abst_test = preprocess_text(title_abst_test)

In [ ]:
# preprocess
train["title_abst_pre"] = title_abst_train
test["title_abst_pre"] = title_abst_test

In [ ]:
import pycld2 as cld2

# titleの単語数が3以下のものは除外してみる
train["title_word_len"] = train["title"].str.split(" ").str.len()
train = train[train["title_word_len"]>3]

# titleが他言語の場合は除外
train["title_lang"] = train["title"].fillna("").map(lambda x: cld2.detect(x)[2][0][1])
train = train[(train["title_lang"]=="en")|(train["title_lang"]=="un")]

In [ ]:
train.to_pickle(input_dir+"train_pre_2.pkl")
test.to_pickle(input_dir+"test_pre_2.pkl")

# 単語特徴量行列の作成

In [ ]:
train = pd.read_pickle(input_dir+"train_pre_2.pkl")
test = pd.read_pickle(input_dir+"test_pre_2.pkl")

In [ ]:
train.head()

,id,title,abstract,judgement,title_abst,title_abst_pre,title_word_len,title_lang
0,0,One-year age changes in MRI brain volumes in o...,Longitudinal studies indicate that declines in...,0,One-year age changes in MRI brain volumes in o...,"[year, age, change, mri, brain, volume, old, a...",10,en
1,1,Supportive CSF biomarker evidence to enhance t...,The present study was undertaken to validate t...,0,Supportive CSF biomarker evidence to enhance t...,"[supportive, csf, biomarker, evidence, enhance...",23,en
2,2,Occurrence of basal ganglia germ cell tumors w...,Objective: To report a case series in which ba...,0,Occurrence of basal ganglia germ cell tumors w...,"[occurrence, basal, ganglia, germ, cell, tumor...",10,en
3,3,New developments in diagnosis and therapy of C...,The etiology and pathogenesis of idiopathic ch...,0,New developments in diagnosis and therapy of C...,"[new, development, diagnosis, therapy, crohn, ...",13,en
4,4,Prolonged shedding of SARS-CoV-2 in an elderly...,NaN,0,Prolonged shedding of SARS-CoV-2 in an elderly...,"[prolong, shedding, sars, cov, elderly, liver,...",16,en


In [ ]:
train_1 = train[train.judgement == 1]
positive_ids = list(train_1["id"])

In [ ]:
#train_0 = train[train.judgement == 0].reset_index(drop=True)
#train_1 = train[train.judgement == 1].reset_index(drop=True)


#train_all = pd.concat([train_1,train_0]).reset_index(drop=True)
#train_all["id"] = train_all.index

In [ ]:
len(train[train.judgement == 1])

626

In [ ]:
#ワードカウント
title_abst_pre = train.title_abst_pre 
words_count = {}

for words in title_abst_pre:
  for word in words:
    if not word in words_count:
      words_count[word] = 1
    else:
      words_count[word] += 1  

# データフレームにする
df_words_count = pd.DataFrame(words_count.values(), index=words_count.keys(), columns=['count'])

# 確認
df_words_count

,count
year,8277
age,9459
change,6872
mri,4439
brain,13314
...,...
microcollimator,1
robotisation,1
ambit,1
pitfallthe,1


In [ ]:
df_words_count.sort_values('count', ascending=False)

,count
patient,54398
covid,39851
disease,32454
study,22563
use,19863
...,...
purposively,1
travailbackground,1
contre,1
preventives,1


In [ ]:
# 単語のリストから、3回以下を削除
df_words_count_2 = df_words_count[df_words_count['count'] >= 3].copy()
all_words = list(df_words_count_2.index)

# ソート
all_words_sort = sorted(all_words)
all_words_sort

len(all_words_sort)

26653

In [ ]:
vecs_words = []

for doc_words in title_abst_pre:
  word_count = [1 if word in doc_words else 0 for word in all_words_sort]
  vecs_words.append(word_count)

In [ ]:
# データフレームへ変換
df_feature_word = pd.DataFrame(vecs_words, columns=all_words_sort)

In [ ]:
# 各単語をカラムに持つ特徴量行列を作成
df_feature_word

,-PRON-,aa,aaa,aacd,aacr,aad,aada,aae,aagp,aai,aami,aan,aao,aap,aapm,aas,aat,aatf,aav,aavp,ab,aba,abacavir,abad,abalphac,abandon,abasic,abate,abatement,abbott,abbreviate,abbreviation,abbvie,abc,abca,abcb,abcc,abcd,abcg,abcs,...,zhuhai,zidovudine,ziehl,zika,zikv,zinc,zingiberis,zip,zipper,zj,zl,zla,zlb,zmk,zn,znf,znpc,znt,zoledronate,zona,zonal,zone,zones,zoning,zooepidemicus,zoom,zoonosis,zoonotic,zoster,zr,zubal,zumba,zung,zunyi,zurich,zusanli,zwitterionic,zymogen,zymography,zymosan
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26916,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26917,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26918,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26919,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_feature_word.to_pickle(input_dir+"frature_word_df_2.pkl")

# 類似度行列の作成

In [ ]:
#cos類似度による類似度行列を作成
doc_sim = cosine_similarity(df_feature_word.values)
df_doc_sim = pd.DataFrame(doc_sim, index=df_feature_word.index, columns=df_feature_word.index)

# 対角成分を0にする
for i in range(len(df_doc_sim)):
  df_doc_sim.iloc[i, i] = 0

df_doc_sim.columns = train["id"]
df_doc_sim.index = train["id"]

In [ ]:
# 確認
df_doc_sim

id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,27105,27106,27107,27108,27109,27110,27111,27112,27113,27114,27115,27116,27117,27118,27119,27120,27121,27122,27123,27124,27125,27126,27127,27128,27129,27130,27131,27132,27133,27134,27135,27136,27137,27138,27139,27140,27141,27142,27143,27144
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.113961,0.154997,0.086763,0.032075,0.088889,0.028931,0.091287,0.089087,0.248217,0.0,0.074074,0.103331,0.033501,0.236914,0.142393,0.064815,0.062439,0.057378,0.020286,0.078567,0.065932,0.000000,0.113228,0.122838,0.000000,0.055835,0.052058,0.034021,0.144444,0.103331,0.075125,0.095783,0.111803,0.000000,0.062505,0.030179,0.153978,0.121018,0.046136,...,0.102930,0.065689,0.083387,0.029814,0.056999,0.015262,0.047926,0.086066,0.083918,0.0,0.041266,0.033501,0.109764,0.054217,0.176896,0.062715,0.067344,0.132569,0.058561,0.110560,0.104805,0.056701,0.260579,0.075544,0.052970,0.058238,0.107833,0.092962,0.121018,0.000000,0.072528,0.108465,0.078170,0.143777,0.095299,0.024103,0.018781,0.000000,0.061728,0.069130
1,0.113961,0.000000,0.092734,0.062292,0.032898,0.079772,0.059346,0.145644,0.121829,0.132826,0.0,0.075974,0.092734,0.051541,0.161993,0.204462,0.227921,0.128081,0.088273,0.083225,0.126629,0.081148,0.000000,0.193552,0.194709,0.000000,0.114535,0.071191,0.023262,0.113961,0.066238,0.077051,0.174647,0.089188,0.000000,0.141037,0.092858,0.133631,0.169257,0.085175,...,0.105569,0.089831,0.061089,0.050965,0.081845,0.078268,0.061443,0.161835,0.114760,0.0,0.042324,0.068721,0.112579,0.088971,0.238130,0.096485,0.069071,0.123608,0.180187,0.170093,0.097720,0.093048,0.218668,0.166032,0.086926,0.119463,0.270351,0.136209,0.124122,0.000000,0.074388,0.123608,0.103081,0.147464,0.097743,0.123608,0.038526,0.073947,0.063311,0.094537
2,0.154997,0.092734,0.000000,0.118006,0.100673,0.104623,0.105939,0.053060,0.103562,0.135492,0.0,0.129164,0.040541,0.052575,0.129835,0.148976,0.096873,0.097988,0.120060,0.063671,0.105685,0.055184,0.000000,0.144787,0.093467,0.058124,0.105150,0.072619,0.047458,0.104623,0.067568,0.137546,0.077942,0.051988,0.036761,0.091552,0.105246,0.074352,0.069061,0.057923,...,0.078319,0.045817,0.049852,0.103975,0.059634,0.063871,0.050141,0.075038,0.078042,0.0,0.107933,0.035050,0.025520,0.068068,0.080970,0.174971,0.199627,0.063044,0.098029,0.080970,0.139554,0.094916,0.136312,0.079037,0.044335,0.134047,0.050141,0.138943,0.184164,0.036761,0.043361,0.075653,0.081783,0.125353,0.274187,0.088262,0.039299,0.018858,0.129164,0.132598
3,0.086763,0.062292,0.118006,0.000000,0.022542,0.140556,0.071162,0.085540,0.076522,0.060676,0.0,0.086763,0.090774,0.070632,0.110999,0.140100,0.130145,0.087762,0.040324,0.057027,0.118319,0.111206,0.000000,0.132624,0.125568,0.039043,0.070632,0.146341,0.063758,0.109322,0.081697,0.013199,0.104711,0.043652,0.049386,0.096640,0.148463,0.083241,0.108245,0.077817,...,0.072337,0.061553,0.075346,0.083812,0.072104,0.064356,0.075783,0.100810,0.085188,0.0,0.029001,0.070632,0.042856,0.060964,0.062159,0.132224,0.102544,0.101637,0.065849,0.069929,0.133918,0.111576,0.074917,0.083429,0.111679,0.130972,0.117885,0.074665,0.115976,0.024693,0.087380,0.152455,0.102024,0.092624,0.033487,0.067758,0.039597,0.076004,0.069411,0.089070
4,0.032075,0.032898,0.100673,0.022542,0.000000,0.028868,0.150329,0.105409,0.102869,0.000000,0.0,0.096225,0.000000,0.043519,0.029311,0.147979,0.096225,0.054074,0.074536,0.105409,0.058321,0.000000,0.080064,0.065372,0.029013,0.288675,0.145065,0.090167,0.058926,0.028868,0.134231,0.243975,0.055300,0.032275,0.182574,0.032478,0.026135,0.061546,0.028583,0.000000,...,0.024311,0.000000,0.030949,0.077460,0.088852,0.079305,0.000000,0.000000,0.024225,0.0,0.214423,0.261116,0.095059,0.028172,0.028724,0.162938,0.058321,0.062622,0.030429,0.028724,0.049507,0.058926,0.030773,0.140193,0.000000,0.060523,0.031129,0.069007,0.142915,0.273861,0.026919,0.062622,0.029013,0.155643,0.092848,0.000000,0.000000,0.0000

In [ ]:
df_doc_sim.to_pickle(input_dir+"sim_df_2.pkl")

# Positive_idに最も類似する文章を抜き出す

In [ ]:
def get_similar_doc(doc_id, df_doc_sim, top_n):
    doc_ids = list(df_doc_sim[doc_id].sort_values(ascending=False)[:top_n].index)
    return doc_ids

In [ ]:
id_list=[]

for positive_id in positive_ids:
    ids = get_similar_doc(positive_id, df_doc_sim, top_n = 1)
    id_list.append(ids)

In [ ]:
def flatten(nested_list):
    result= [e for inner_list in nested_list for e in inner_list]
    result = set(result)
    result = list(set(result) - set(positive_ids))
    return result

id_list = flatten(id_list)

In [ ]:
len(id_list)

228

In [ ]:
def filter_id(x):
    if x in id_list:
        x=1
        return x
    else:
        x=0
        return x

train["judgement_1"] = train["id"].apply(lambda x: filter_id(x))

In [ ]:
train[train["judgement_1"] == 1]

,id,title,abstract,judgement,title_abst,title_abst_pre,title_word_len,title_lang,judgement_1
48,48,The keys to control a COVID-19 outbreak in a h...,Background: The high rate of severe acute resp...,0,The keys to control a COVID-19 outbreak in a h...,"[key, control, covid, outbreak, haemodialysis,...",11,en,1
74,74,White matter hyperintensities are positively a...,White matter hyperintensities are associated w...,0,White matter hyperintensities are positively a...,"[white, matter, hyperintensitie, positively, a...",12,en,1
118,118,Clinical Features of 154 COVID-19 Patients and...,Objective We aimed to clarify clinical and lab...,0,Clinical Features of 154 COVID-19 Patients and...,"[clinical, feature, covid, patient, parameter,...",24,en,1
182,182,Screening with HRCT chest and PCR testing for ...,NaN,0,Screening with HRCT chest and PCR testing for ...,"[screen, hrct, chest, pcr, test, covid, asympt...",18,en,1
220,220,Decreased CSF amyloid beta42 and normal tau le...,NaN,0,Decreased CSF amyloid beta42 and normal tau le...,"[decreased, csf, amyloid, beta, normal, tau, l...",13,en,1
...,...,...,...,...,...,...,...,...,...
25916,25916,Hydrocolonic sonography: A novel screening met...,Hydrocolonic sonography is an easy and harmles...,0,Hydrocolonic sonography: A novel screening met...,"[hydrocolonic, sonography, novel, screening, m...",12,en,1
25918,25918,Imaging infection with 18F-FDG-labeled leukocy...,The aim of this study was to assess the feasib...,0,Imaging infection with 18F-FDG-labeled leukocy...,"[imaging, infection, f, fdg, label, leukocyte,...",11,en,1
26054,26054,All-in-One Dual CRISPR-Cas12a (AIOD-CRISPR) As...,A recent outbreak of novel coronavirus (SARS-C...,0,All-in-One Dual CRISPR-Cas12a (AIOD-CRISPR) As...,"[dual, crispr, cas, aiod, crispr, assay, case,...",20,en,1
26292,26292,Analysis of adjunctive serological detection t...,Background: The severe acute respiratory syndr...,0,Analysis of adjunctive serological detection t...,"[analysis, adjunctive, serological, detection,...",19,en,1


In [ ]:
train["judgement"]=train["judgement"]+train["judgement_1"]
train_pre = train

In [ ]:
train_pre = train_pre.iloc[:,:-1]
train_pre

,id,title,abstract,judgement,title_abst,title_abst_pre,title_word_len,title_lang
0,0,One-year age changes in MRI brain volumes in o...,Longitudinal studies indicate that declines in...,0,One-year age changes in MRI brain volumes in o...,"[year, age, change, mri, brain, volume, old, a...",10,en
1,1,Supportive CSF biomarker evidence to enhance t...,The present study was undertaken to validate t...,0,Supportive CSF biomarker evidence to enhance t...,"[supportive, csf, biomarker, evidence, enhance...",23,en
2,2,Occurrence of basal ganglia germ cell tumors w...,Objective: To report a case series in which ba...,0,Occurrence of basal ganglia germ cell tumors w...,"[occurrence, basal, ganglia, germ, cell, tumor...",10,en
3,3,New developments in diagnosis and therapy of C...,The etiology and pathogenesis of idiopathic ch...,0,New developments in diagnosis and therapy of C...,"[new, development, diagnosis, therapy, crohn, ...",13,en
4,4,Prolonged shedding of SARS-CoV-2 in an elderly...,NaN,0,Prolonged shedding of SARS-CoV-2 in an elderly...,"[prolong, shedding, sars, cov, elderly, liver,...",16,en
...,...,...,...,...,...,...,...,...
27140,27140,The amyloidogenic pathway of amyloid precursor...,Amyloid beta-protein (A beta) is the main cons...,0,The amyloidogenic pathway of amyloid precursor...,"[amyloidogenic, pathway, amyloid, precursor, p...",15,en
27141,27141,Technologic developments in radiotherapy and s...,We present a review of current technological p...,0,Technologic developments in radiotherapy and s...,"[technologic, development, radiotherapy, stere...",7,en
27142,27142,Novel screening cascade identifies MKK4 as key...,Phosphorylation of Tau at serine 422 promotes ...,0,Novel screening cascade identifies MKK4 as key...,"[novel, screening, cascade, identify, mkk, key...",13,en
27143,27143,Visualization of the gall bladder on F-18 FDOP...,The ability to label dihydroxyphenylalanine (D...,0,Visualization of the gall bladder on F-18 FDOP...,"[visualization, gall, bladder, f, fdopa, pet, ...",13,en


In [ ]:
len(train_pre[train_pre.judgement == 1])

854

In [ ]:
train_pre.to_pickle(input_dir+"train_rev_2.pkl")